<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/MTATFP_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MTATFP trained by original author using model saved in their github, here set as GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pubchempy --quiet
!pip install transformers --quiet
!pip install cairosvg --quiet
!pip install varname --quiet
!pip install Cython --quiet
!pip install rdkit --quiet
# !pip install molsets --quiet
!pip install pathlib --quiet
!pip install xgboost==1.6.1 --quiet
!pip install dgllife --quiet
!pip install molvs --quiet

# !pip install dgl==1.1 --quiet # cpu version, usable for calculation

!pip uninstall dgl -y # dgl cuda version for training using gpu
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

!python --version
import torch
print('torch version: ', torch.__version__)
print('cuda available: ', torch.cuda.is_available())
import dgl
print('dgl version: ', dgl.__version__)
import dgllife
print('dgllife version: ', dgllife.__version__)
import rdkit
print('rdkit version: ', rdkit.__version__)
import molvs
print('molvs version: ', molvs.__version__)
import matplotlib
print('matplotlib version: ', matplotlib.__version__)

In [1]:
cd /content/drive/MyDrive/A_JAK_design

/content/drive/MyDrive/A_JAK_design


In [2]:
from help_function.function import *
from help_function.jak_dataset import *

In [3]:
from MTATFP.MTATFP_predict import *

In [6]:
data_path = '/content/drive/MyDrive/A_JAK_design/Data/data_label_0.25/'
for enzyme in ['JAK1']:
    for file_type in ['valid']:
        file_name = enzyme + '_' + file_type + '.csv'
        data = pd.read_csv(data_path + file_name)
        # df = MTATFP_predict(data['SMILES'])
        # print(data['SMILES'])
_, binary_df = MTATFP_predict(data['SMILES'], convert_binary=True)
enzymes = ['JAK1', 'JAK2', 'JAK3', 'TYK2']
l = ['SMILES'] + [enzyme +'_pred' for enzyme in enzymes]
binary_df[l]

bins/  folder is in directory:  True
Processing dgl graphs from scratch...
ExTest sets:  874


['SMILES', 'JAK1_pred', 'JAK2_pred', 'JAK3_pred', 'TYK2_pred']

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# from torch.utils.data import DataLoader
# from dgllife.model import model_zoo
# from dgllife.utils import smiles_to_bigraph
# from dgllife.utils import AttentiveFPAtomFeaturizer
# from dgllife.utils import AttentiveFPBondFeaturizer
# from dgllife.data import MoleculeCSVDataset
# import dgl
# import matplotlib
# import matplotlib.cm as cm
# from IPython.display import SVG, display
# from rdkit import Chem
# from rdkit.Chem import rdDepictor
# from rdkit.Chem.Draw import rdMolDraw2D
# from datetime import datetime
# import os
# import math

# if torch.cuda.is_available():
#     # print('use GPU')
#     device = 'cuda'
# else:
#     # print('use CPU')
#     device = 'cpu'


# def handle_less_symbol(a_string):
# #     l = a_string.split()
#     s = a_string[1:]
#     try:
#         s = int(s)
#         return s
#     except:
#         print('cannot convert s')
#         return s

# def threshold_IC50(row, enzyme, thres=250):
#     num = row[enzyme]
#     if isinstance(num, str):
#         try:
#             num = float(num)
#         except: num = handle_less_symbol(num)
#     try:
#         if num > thres: return 0 # noninhibitor
#         else: return 1
#     except: # <500 type, cannot convert
#         print('error', num)

# # !cd /Users/uranaieiko_1/Desktop/JAK_design/MTATFP/Data


# def create_path(path):
#     # Check whether the specified path exists or not
#     isExist = os.path.exists(path)
#     #printing if the path exists or not
#     # print(path, ' folder is in directory: ', isExist)
#     if not isExist:
#     # Create a new directory because it does not exist
#         os.makedirs(path)
#         print(path, " is created!")

# def collate_molgraphs(data):
#     assert len(data[0]) in [3, 4], \
#         'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
#     if len(data[0]) == 3:
#         smiles, graphs, labels = map(list, zip(*data))
#         masks = None
#     else:
#         smiles, graphs, labels, masks = map(list, zip(*data))

#     bg = dgl.batch(graphs)
#     bg.set_n_initializer(dgl.init.zero_initializer)
#     bg.set_e_initializer(dgl.init.zero_initializer)
#     labels = torch.stack(labels, dim=0)

#     if masks is None:
#         masks = torch.ones(labels.shape)
#     else:
#         masks = torch.stack(masks, dim=0)
#     return smiles, bg, labels, masks


# def MTATFP_predict(df_smiles, savename=None, save_path='prediction/',
#                    convert_binary=False, thres_CoGT=250,
#                    model_file='model/MTATFP_jak.pt'):
#     now = datetime.now() # current date and time
#     date_time = now.strftime("%m_%d_%Y__%H:%M:%S")
#     bin_path = 'bins/'
#     create_path(bin_path)
#     cache_file_path = bin_path + date_time + '.bin'

#     atom_featurizer = AttentiveFPAtomFeaturizer(atom_data_field='hv')
#     bond_featurizer = AttentiveFPBondFeaturizer(bond_data_field='he')
#     n_feats = atom_featurizer.feat_size('hv')
#     e_feats = bond_featurizer.feat_size('he')

#     data_df = pd.DataFrame()
#     data_df['SMILES'] = df_smiles

#     for task_name in ['pIC50_JAK1','pIC50_JAK2','pIC50_JAK3','pIC50_TYK2']:
#         data_df[task_name] = 888888888888888 # Random num

#     extest_datasets = MoleculeCSVDataset(data_df,
#                             smiles_to_graph=smiles_to_bigraph,
#                             node_featurizer=atom_featurizer,
#                             edge_featurizer= bond_featurizer,
#                             smiles_column='SMILES',
#                             cache_file_path=cache_file_path,
#                             task_names=['pIC50_JAK1','pIC50_JAK2',
#                                         'pIC50_JAK3','pIC50_TYK2'],
#                             load=True,init_mask=True
#                             )

#     print('ExTest sets: ',len(extest_datasets))

#     fn = model_file
#     model = model_zoo.AttentiveFPPredictor(node_feat_size=n_feats,
#                                     edge_feat_size=e_feats,
#                                     num_layers=2,
#                                     num_timesteps=1,
#                                     graph_feat_size=300,
#                                     n_tasks=4,
#                                         )
#     model.load_state_dict(torch.load(fn,map_location=torch.device(device)))

#     for i in range(len(data_df)):
#         smiles, g, label, _ = extest_datasets[i]
#         g = dgl.batch([g])
#         atom_feats, bond_feats = g.ndata.pop('hv'), g.edata.pop('he')
#         preds, _ = model(g, atom_feats, bond_feats, get_node_weight=True)
#         jak_list = [round(10**(-1.0 * i), 4) for i in preds[0].tolist()]
#         for enzyme, ic50 in zip(['JAK1', 'JAK2', 'JAK3', 'TYK2'], jak_list):
#             data_df.loc[i, enzyme] = ic50
#     # print(data_df[['SMILES', 'JAK1', 'JAK2', 'JAK3', 'TYK2']])
#     data_df = data_df[['SMILES', 'JAK1', 'JAK2', 'JAK3', 'TYK2']]
#     if savename != None:
#         # save_path = 'prediction/'
#         create_path(save_path)
#         print('Data saved in ', save_path + savename)
#         data_df.to_csv(save_path+savename, index=False)
#     if convert_binary == True:
#         new_df = pd.DataFrame()
#         for enzyme in ['JAK1', 'JAK2', 'JAK3', 'TYK2']:
#             new_df['SMILES'] = data_df['SMILES']
#             new_df[enzyme] = data_df[enzyme]
#             new_df[enzyme+'_pred'] = data_df.apply(
#                     lambda row: threshold_IC50(row, enzyme, thres_CoGT), axis=1)
#         print(f'Data processed for CoGT binary ({thres_CoGT}nM) saved in ',
#                   save_path+f'for_CoGT_{thres_CoGT}nM_'+savename)
#         new_df.to_csv(save_path+f'for_CoGT_{thres_CoGT}nM_'+savename,
#                       index=False)
#     return data_df

# # EXAMPLE
# # Only need Smiles column
# # MTATFP_pred(data_test['SMILES'])

# # def MTATFP_pred(df_smiles, savename=None, save_path='prediction/',
# #                 convert_binary=False, thres_CoGT=250):

# # model_path = '../CoGT_0.25uM/model_0.25nM_data_train_same_MTATFP/'
# # data_path = '../CoGT_0.25uM/Data_MTATFP/data_label_0.25/'
# # save_path_MTATFP = data_path +'pred_MTATFP/'

# # data_path_MACCS = data_path + 'MACCS/'
# # from os import walk
# for enzyme in ['JAK1', 'JAK2', 'JAK3', 'TYK2']:
#     for file_type in ['valid', 'test']:
#         file_name = enzyme + '_' + file_type + '.csv'
#         data = pd.read_csv(data_path + file_name)
#         df = MTATFP_predict(data['SMILES'])
#         # print(data['SMILES'])


